# Findings 
Here you find a summary of the initial findings

## Some stats

In [2]:
import pandas as pd
df = pd.read_csv('../data/archaeology_metadata.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158254 entries, 0 to 158253
Data columns (total 13 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   dsPersistentId                                        158254 non-null  object 
 1   publicationStatus                                     158254 non-null  object 
 2   title                                                 158254 non-null  object 
 3   dsDescriptionValue                                    158254 non-null  object 
 4   dansSpatialPointX                                     56479 non-null   object 
 5   dansSpatialPointY                                     56479 non-null   object 
 6   dansSpatialPointScheme                                56304 non-null   object 
 7   dansSpatialBoxNorth                                   4432 non-null    object 
 8   dansSpatialBoxEast                          

In total, the archaeology data station contains 158254 published datasets. 
About 60000 have geospatial information in the metadata, either in the form of points (x, y), or a bounding box. 

--> About* 1/3 of the datasets have spatial metadata available. 

### Inspect the data with missing explicit geospatial metadata

In [4]:
df_nogeo = df[df.dansSpatialBoxNorth.isna() & df.dansSpatialPointX.isna()]

In [5]:
df_nogeo

,dsPersistentId,publicationStatus,title,dsDescriptionValue,dansSpatialPointX,dansSpatialPointY,dansSpatialPointScheme,dansSpatialBoxNorth,dansSpatialBoxEast,dansSpatialBoxSouth,dansSpatialBoxWest,dansSpatialBoxScheme,dansSpatialCoverageControlleddansSpatialCoverageText
109,doi:10.17026/dans-246-zkdb,Published,Rotterdam Noordsingel 90-92,onderzoeksrapport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,doi:10.17026/dans-zaz-bggb,Published,Verslag van een verkennend booronderzoek te Ge...,onderzoeksrapport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,doi:10.17026/dans-xgk-zzky,Published,Verslag van een verkennend booronderzoek op de...,onderzoeksrapport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,doi:10.17026/dans-zaa-rsht,Published,Nesserlaan 2 te Witmarsum,Aan het plangebied was een onbekend verwachtin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,doi:10.17026/dans-x8b-vxz8,Published,"Plangebieden N670 Yerseke, N670 Kloetinge en N...",Zoals op basis van het bureauonderzoek werd ve...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
158210,doi:10.17026/AR/VEIR0Z,Published,Archeologische Begeleiding Wind A16 kabeltracé...,Tussen oktober 2021 en juni 2022 is door Antea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158216,doi:10.17026/AR/WUTNMU,Published,"Oudemolen, Groot Boerenbos (Gemeente Tynaarlo,...",Het inventariserend booronderzoek is in twee f...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158232,doi:10.17026/AR/XZ57NT,Published,Archeologisch Bureauonderzoek. Burgemeester va...,Archeologisch bureauonderzoek naar de aanwezig...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158233,doi:10.17026/AR/TAXDID,Published,Archeologisch Bureauonderzoek. Twee tracés per...,Archeologisch Bureauonderzoek naar de aanwezig...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Possibly useful variables 

| variable | description | example | source doi |
|---|---|---|---|
| dansTemporalSpatial:dansSpatialPoint | Has an X, Y and Scheme specification | 6.11557, 52.78667, longitude/latitude (degrees) |  10.17026/25AR/25AAV0BL | 
| dansTemporalSpatial:dansSpatialCoverageControlled |  | Netherlands | 10.17026/25AR/25AAV0BL | 
| dansTemporalSpatial:dansSpatialCoverageText | | 	"Sint-Michielsgestel", "NLD" | 10.17026/AR/KDIBWJ | 
| dansRelationMetadata:dansCollection:title | | Kalverstraat 11 Steenwijk | 10.17026/25AR/25AAV0BL | 
| ore:aggregates:schema:name | Contains the name of a file, often with location information | 115-261_Steenwijk_Kalverstraat 11.pdf / location.xml |  10.17026/25AR/25AAV0BL \ 10.17026/AR/KDIBWJ | 
| dansRelationMetadata:dansCollection:title | Title of the dataset, can contain clues on geospatial info |  Kalverstraat 11 Steenwijk | 10.17026/25AR/25AAV0BL | 
| citation:dsDescriptionValue | Contains a free description of the dataset | | | 
| ore:describes:language | Language of the report/dataset, might give a clue in toponym disambiguation step |  | | 




### Location files 
- This [location.xml](https://archaeology.datastations.nl/file.xhtml?fileId=3746676&version=1.0) is restricted